#### Як змінювався середній рейтинг фільмів в залежності від мови за останні 5 років порівняно з середнім рейтингом цих фільмів за весь час?



In [2]:
from data_preparation.title_principals_extract_transform import \
    title_principals_extract_transform
from data_preparation.name_basics_extract_transform import \
    name_basics_extract_transform
from data_preparation.title_akas_extract_transform import \
    title_akas_extract_transform
from data_preparation.title_ratings_extract_transform import \
    title_ratings_extract_transform
from data_preparation.title_basics_extract_transform import \
    title_basics_extract_transform
from business_queries.query_ufimtseva import get_titles_by_region, \
    compute_language_rating_trends, get_top_rated_alternative_titles, \
    actors_in_high_rated_movies, movies_quartiles_by_region, \
    translations_by_region
from pyspark.sql import SparkSession

name_basics_path = "../data/name.basics.tsv"
title_akas_path = "../data/title.akas.tsv"
title_basics_path = "../data/title.basics.tsv"
title_crew_path = "../data/title.crew.tsv"
title_episode_path = "../data/title.episode.tsv"
title_principals_path = "../data/title.principals.tsv"
title_ratings_path = "../data/title.ratings.tsv"

spark = SparkSession.builder \
        .appName("IMDB Data Analysis") \
        .config("spark.driver.memory", "12g") \
        .getOrCreate()
name_basics_df = name_basics_extract_transform(spark, name_basics_path)
title_akas_df = title_akas_extract_transform(spark, title_akas_path)
title_basics_df = title_basics_extract_transform(spark, title_basics_path)
title_principals_df = title_principals_extract_transform(spark, title_principals_path)
title_ratings_df = title_ratings_extract_transform(spark, title_ratings_path)


In [4]:

language_rating_trends_df = compute_language_rating_trends(
    title_basics_df,
    title_ratings_df,
    title_akas_df
)
language_rating_trends_df.show(truncate=False, n=10)

+--------+-----------------------+-------------------+---------------------+
|language|Avg_Rating_Last_5_Years|Avg_Rating_All_Time|Rating_Difference    |
+--------+-----------------------+-------------------+---------------------+
|en      |6.388755585450938      |6.41024701072111   |-0.021491425270172115|
|ro      |7.300000190734863      |6.260869549668354  |1.0391306410665093   |
|uz      |4.400000095367432      |6.738235270275789  |-2.338235174908357   |
|ur      |6.555999965667724      |6.613888874981138  |-0.05788890931341406 |
|pl      |4.6000001430511475     |5.850000023841858  |-1.2499998807907104  |
|jsl     |6.400000095367432      |6.400000095367432  |0.0                  |
|yi      |5.3                    |6.408510634239684  |-1.1085106342396838  |
|sk      |6.719999980926514      |6.919924103557724  |-0.1999241226312103  |
|hy      |6.5                    |6.057142862251827  |0.4428571377481729   |
|kk      |7.539999961853027      |6.6698630346010805 |0.8701369272519468   |

#### Які локалізовані (альтернативні) назви фільмів і телевізійних шоу за останні 5 років отримали найвищі оцінки, і як вони ранжуються в кожному регіоні за середнім рейтингом?

In [9]:
alternative_titles = get_top_rated_alternative_titles(title_akas_df, title_ratings_df, title_basics_df)
alternative_titles.show(truncate=False, n=20)

+----------------------------------------+------+-------------+--------------+
|title                                   |region|averageRating|rank_by_rating|
+----------------------------------------+------+-------------+--------------+
|The Oath of Cyriac                      |AD    |9.5          |1             |
|Tirites                                 |AD    |8.8          |2             |
|Mr. Hand Solo                           |AD    |7.0          |3             |
|Quiet                                   |AD    |7.0          |3             |
|Under the Ice                           |AD    |6.6          |5             |
|Frialdad (Coldness)                     |AD    |5.7          |6             |
|Dhoora - Always Together                |AE    |10.0         |1             |
|Marasim - Beyond Borders                |AE    |10.0         |1             |
|Bear: Part 1                            |AE    |9.9          |3             |
|Wild Abu Dhabi: The Turtles of Al Dhafra|AE    |9.9

#### Які актори найчастіше асоціюються з фільмами з високим рейтингами, і як варіюється їхня участь залежно від регіону?

In [10]:
actors_in_high_rated_movies_df = actors_in_high_rated_movies(title_akas_df, title_ratings_df, title_principals_df, name_basics_df)
actors_in_high_rated_movies_df.show(truncate=False, n=20)

+--------------------+------+-----------+
|actor_name          |region|appearances|
+--------------------+------+-----------+
|Dee Bradley Baker   |US    |1995       |
|Faye Peters         |DE    |1410       |
|Faye Peters         |IT    |1410       |
|Faye Peters         |PT    |1410       |
|Faye Peters         |FR    |1409       |
|Faye Peters         |JP    |1409       |
|Faye Peters         |IN    |1409       |
|Faye Peters         |ES    |1409       |
|Dan Green           |JP    |1327       |
|Colleen Clinkenbeard|JP    |1090       |
|Fred Rogers         |US    |1070       |
|Grey Griffin        |US    |1052       |
|Frank Welker        |US    |944        |
|Tom Kenny           |US    |943        |
|Genevieve Howard    |FR    |938        |
|Genevieve Howard    |IT    |938        |
|Genevieve Howard    |JP    |938        |
|Genevieve Howard    |IN    |938        |
|Genevieve Howard    |ES    |938        |
|Genevieve Howard    |DE    |938        |
+--------------------+------+-----

#### У яких регіонах за останні 5 років було випущено найбільше оригінальних творів, і чи є зв’язок між їх кількістю та середніми оцінками?

In [11]:
titles_by_region_df = get_titles_by_region(title_akas_df, title_basics_df, title_ratings_df)
titles_by_region_df.show(truncate=False, n=20)

+------+-------------------+------------------+
|region|num_original_titles|avg_rating        |
+------+-------------------+------------------+
|IN    |857337             |6.998960083706314 |
|DE    |848267             |6.94235402394119  |
|FR    |830366             |7.088119805516488 |
|JP    |829302             |7.113833878305049 |
|ES    |820510             |7.124436371773924 |
|IT    |818101             |7.149352735232591 |
|PT    |797434             |7.380931231334436 |
|US    |269623             |6.700586325644089 |
|GB    |89920              |6.450516264214483 |
|CA    |76062              |6.4207843163579525|
|AU    |45419              |6.362385292588854 |
|XWW   |29990              |6.599783394187919 |
|PH    |24115              |6.1725880716792   |
|RU    |21816              |6.2763588144539595|
|BR    |21147              |6.394928866825501 |
|MX    |20328              |6.45109818535718  |
|PL    |18235              |6.25512916352547  |
|CN    |14700              |6.6747206567

#### Які фільми мають найвищі середні рейтинги в кожному регіоні, якщо розподілити їх за квартилями?

In [12]:
movies_quartiles_by_region_df = movies_quartiles_by_region(title_akas_df, title_basics_df, title_ratings_df)
movies_quartiles_by_region_df.show(truncate=False, n=20)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+------+-------------+---------------+
|primaryTitle                                                                                                                                             |region|averageRating|rating_quartile|
+---------------------------------------------------------------------------------------------------------------------------------------------------------+------+-------------+---------------+
|The Oath of Cyriac                                                                                                                                       |AD    |9.5          |1              |
|Tirites                                                                                                                                                  |AD    |8.8          |1              |
|Impacto                           

#### У яких регіонах фільми мають найбільше перекладених назв?

In [13]:
translations_by_region_df = translations_by_region(title_akas_df, title_basics_df)
translations_by_region_df.show(truncate=False, n=20)

+-------------+------+----------------+
|originalTitle|region|num_translations|
+-------------+------+----------------+
|Episode #1.1 |FR    |50405           |
|Episode #1.1 |JP    |50390           |
|Episode #1.1 |IT    |50389           |
|Episode #1.1 |ES    |50389           |
|Episode #1.1 |PT    |50389           |
|Episode #1.1 |DE    |50389           |
|Episode #1.1 |IN    |50389           |
|Episode #1.2 |FR    |45784           |
|Episode #1.2 |PT    |45763           |
|Episode #1.2 |JP    |45763           |
|Episode #1.2 |IT    |45763           |
|Episode #1.2 |DE    |45763           |
|Episode #1.2 |IN    |45763           |
|Episode #1.2 |ES    |45763           |
|Episode #1.3 |FR    |43288           |
|Episode #1.3 |ES    |43269           |
|Episode #1.3 |JP    |43268           |
|Episode #1.3 |IT    |43268           |
|Episode #1.3 |DE    |43268           |
|Episode #1.3 |IN    |43268           |
+-------------+------+----------------+
only showing top 20 rows

